Modifierad kod modifierad kod från första exemplet, nu med 9000% fler träd!

In [1]:
# For Python 2 / 3 compatability
from __future__ import print_function
import random
import operator

In [2]:
# Toy dataset.
# Format: each row is an example.
# The last column is the label.
# The first two columns are features.
# Feel free to play with it by adding more features & examples.
# Interesting note: I've written this so the 2nd and 5th examples
# have the same features, but different labels - so we can see how the
# tree handles this case.
training_data = [    [54.4, 14.4, 'Kylrum'],    [45.4, 12.4, 'Kylrum'],    [89.4, 19.5, 'Klassrum'],
[57.4, 18.1, 'Lärarrum'],     [22.4, 8.6, 'Kylrum'],    [24.4, 11.24, 'Kylrum'],    [84.4, 24.4,
'Klassrum'],    [95.4, 22.4, 'Klassrum'],    [81.4, 20.1, 'Lärarrum'],    [70, 19.7, 'Lärarrum'],  [70, 19.8,'Klassrum']]

In [3]:
# Column labels.
# These are used only to print the tree.
header = ["luftfuktighet", "temperatur", "label"]

In [4]:
def unique_vals(rows, col):
    """Find the unique values for a column in a dataset."""
    return set([row[col] for row in rows])

In [5]:
#######
# Demo:
unique_vals(training_data, 2)
# unique_vals(training_data, 1)
#######

{'Klassrum', 'Kylrum', 'Lärarrum'}

In [6]:
def class_counts(rows):
    """Counts the number of each type of example in a dataset."""
    counts = {}  # a dictionary of label -> count.
    for row in rows:
        # in our dataset format, the label is always the last column
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

In [7]:
## Här nedan har vi en datagenererare

In [8]:
%%time
def dataGenerator(n):
    data = []
    n1 = random.randint(0, round(n/2)-1)
    n2 = random.randint(0, n-n1)
    n3 = n - n1 - n2
    for x in range(0,n1+1):
        l = round(random.uniform(15,60), 1)
        t = round(random.uniform(10,16), 1)
        label = "Kylrum"
        data.append([l,t,label])
    for x in range(0,n2+1):
        l = round(random.uniform(50,90), 1)
        t = round(random.uniform(16,22), 1)
        label = "Lärarrum"
        data.append([l,t,label])
    for x in range(0,n3+1):
        l = round(random.uniform(80,100), 1)
        t = round(random.uniform(18,24), 1)
        label = "Klassrum"
        data.append([l,t,label])
    return data
training_data = dataGenerator(2000)
random.shuffle(training_data)
training_data

Wall time: 12 ms


[[91.5, 22.3, 'Klassrum'],
 [86.5, 20.7, 'Klassrum'],
 [58.0, 15.7, 'Kylrum'],
 [82.7, 19.8, 'Klassrum'],
 [89.8, 18.4, 'Klassrum'],
 [85.0, 23.6, 'Klassrum'],
 [97.5, 21.4, 'Klassrum'],
 [88.6, 21.5, 'Klassrum'],
 [95.7, 22.8, 'Klassrum'],
 [97.4, 18.4, 'Klassrum'],
 [97.4, 22.6, 'Klassrum'],
 [98.3, 19.2, 'Klassrum'],
 [36.0, 10.2, 'Kylrum'],
 [87.2, 19.6, 'Klassrum'],
 [82.0, 22.7, 'Klassrum'],
 [90.0, 18.9, 'Klassrum'],
 [83.5, 18.6, 'Klassrum'],
 [80.7, 19.0, 'Klassrum'],
 [86.5, 19.2, 'Klassrum'],
 [92.7, 21.8, 'Klassrum'],
 [78.6, 21.9, 'Lärarrum'],
 [92.9, 22.7, 'Klassrum'],
 [100.0, 19.0, 'Klassrum'],
 [90.3, 22.0, 'Klassrum'],
 [78.5, 20.9, 'Lärarrum'],
 [92.4, 23.4, 'Klassrum'],
 [93.1, 18.1, 'Klassrum'],
 [86.7, 22.7, 'Klassrum'],
 [98.6, 20.2, 'Klassrum'],
 [88.8, 18.7, 'Klassrum'],
 [81.7, 23.6, 'Klassrum'],
 [80.1, 19.7, 'Lärarrum'],
 [98.1, 22.2, 'Klassrum'],
 [80.4, 19.2, 'Klassrum'],
 [50.4, 16.2, 'Lärarrum'],
 [70.4, 18.3, 'Lärarrum'],
 [97.2, 21.5, 'Klassrum'],
 [88

In [9]:
#######
# Demo:
class_counts(training_data)
#######

{'Klassrum': 1653, 'Kylrum': 100, 'Lärarrum': 250}

In [10]:
def is_numeric(value):
    """Test if a value is numeric."""
    return isinstance(value, int) or isinstance(value, float)

In [11]:
#######
# Demo:
is_numeric(7)
# is_numeric("Red")
#######

True

In [12]:
class Question:
    """A Question is used to partition a dataset.

    This class just records a 'column number' (e.g., 0 for Color) and a
    'column value' (e.g., Green). The 'match' method is used to compare
    the feature value in an example to the feature value stored in the
    question. See the demo below.
    """

    def __init__(self, column, value, less, eq):
        self.column = column
        self.value  = value
        self.less   = less
        self.eq     = eq

    def match(self, example):
        # Compare the feature value in an example to the
        # feature value in this question.
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value
    def moreless(self, example):
        # Compare the feature value in an example to the
        # feature value in this question.
        val = example[self.column]
        if self.less:
            if self.eq:
                return val <= self.value
            else:
                return val < self.value
        elif is_numeric(self.value):
            if self.eq:
                return val >= self.value
            else:
                return val > self.value
        else:
            return val == self.value

    def __repr__(self):
        # This is just a helper method to print
        # the question in a readable format.
        condition = "=="
        if self.less:
            if self.eq:
                condition = "<="
            else:
                condition = "<"
        elif is_numeric(self.value):
            if self.eq:
                condition = ">="
            else:
                condition = ">"
        return "Är %s %s %s?" % (
            header[self.column], condition, str(self.value))

In [13]:
#######
# Demo:
# Let's write a question for a numeric attribute
Question(1, 19.5, 1, 0)

Är temperatur < 19.5?

In [14]:
# How about one for a categorical attribute
q = Question(0, 50, 1, 0)
q

Är luftfuktighet < 50?

In [15]:
# Let's pick an example from the training set...
example = training_data[0]
# ... and see if it matches the question
q.moreless(example) # this will be true, since the first example is Green.
#######

False

In [16]:
def partition(rows, question):
    """Partitions a dataset.

    For each row in the dataset, check if it matches the question. If
    so, add it to 'true rows', otherwise, add it to 'false rows'.
    """
    true_rows, false_rows = [], []
    for row in rows:
        if question.moreless(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

In [17]:
#######
# Demo:
# Let's partition the training data based on whether rows are Red.
true_rows, false_rows = partition(training_data, Question(1, 19.5, 1, 0))
# This will contain all the 'Red' rows.
true_rows

[[58.0, 15.7, 'Kylrum'],
 [89.8, 18.4, 'Klassrum'],
 [97.4, 18.4, 'Klassrum'],
 [98.3, 19.2, 'Klassrum'],
 [36.0, 10.2, 'Kylrum'],
 [90.0, 18.9, 'Klassrum'],
 [83.5, 18.6, 'Klassrum'],
 [80.7, 19.0, 'Klassrum'],
 [86.5, 19.2, 'Klassrum'],
 [100.0, 19.0, 'Klassrum'],
 [93.1, 18.1, 'Klassrum'],
 [88.8, 18.7, 'Klassrum'],
 [80.4, 19.2, 'Klassrum'],
 [50.4, 16.2, 'Lärarrum'],
 [70.4, 18.3, 'Lärarrum'],
 [81.0, 18.5, 'Lärarrum'],
 [53.3, 17.2, 'Lärarrum'],
 [87.3, 18.8, 'Klassrum'],
 [98.1, 19.1, 'Klassrum'],
 [96.0, 18.1, 'Klassrum'],
 [70.5, 18.3, 'Lärarrum'],
 [84.7, 19.1, 'Klassrum'],
 [94.9, 18.8, 'Klassrum'],
 [91.3, 18.8, 'Klassrum'],
 [72.4, 17.4, 'Lärarrum'],
 [80.6, 17.6, 'Lärarrum'],
 [69.5, 17.3, 'Lärarrum'],
 [16.1, 12.3, 'Kylrum'],
 [98.8, 19.4, 'Klassrum'],
 [94.4, 18.1, 'Klassrum'],
 [27.7, 11.7, 'Kylrum'],
 [40.8, 15.0, 'Kylrum'],
 [55.6, 19.2, 'Lärarrum'],
 [97.2, 19.0, 'Klassrum'],
 [87.7, 18.2, 'Klassrum'],
 [89.9, 18.5, 'Klassrum'],
 [86.3, 18.3, 'Klassrum'],
 [95.0, 18

In [18]:
# This will contain everything else.
false_rows
#######

[[91.5, 22.3, 'Klassrum'],
 [86.5, 20.7, 'Klassrum'],
 [82.7, 19.8, 'Klassrum'],
 [85.0, 23.6, 'Klassrum'],
 [97.5, 21.4, 'Klassrum'],
 [88.6, 21.5, 'Klassrum'],
 [95.7, 22.8, 'Klassrum'],
 [97.4, 22.6, 'Klassrum'],
 [87.2, 19.6, 'Klassrum'],
 [82.0, 22.7, 'Klassrum'],
 [92.7, 21.8, 'Klassrum'],
 [78.6, 21.9, 'Lärarrum'],
 [92.9, 22.7, 'Klassrum'],
 [90.3, 22.0, 'Klassrum'],
 [78.5, 20.9, 'Lärarrum'],
 [92.4, 23.4, 'Klassrum'],
 [86.7, 22.7, 'Klassrum'],
 [98.6, 20.2, 'Klassrum'],
 [81.7, 23.6, 'Klassrum'],
 [80.1, 19.7, 'Lärarrum'],
 [98.1, 22.2, 'Klassrum'],
 [97.2, 21.5, 'Klassrum'],
 [88.0, 20.7, 'Klassrum'],
 [82.8, 22.5, 'Klassrum'],
 [85.5, 22.7, 'Klassrum'],
 [92.2, 19.8, 'Klassrum'],
 [98.5, 21.1, 'Klassrum'],
 [81.1, 21.3, 'Lärarrum'],
 [90.5, 20.6, 'Klassrum'],
 [91.3, 23.9, 'Klassrum'],
 [96.5, 20.7, 'Klassrum'],
 [97.6, 22.7, 'Klassrum'],
 [87.8, 23.6, 'Klassrum'],
 [98.9, 19.9, 'Klassrum'],
 [82.1, 23.8, 'Klassrum'],
 [86.4, 20.3, 'Klassrum'],
 [90.8, 21.8, 'Klassrum'],
 

In [19]:
def gini(rows):
    """Calculate the Gini Impurity for a list of rows.

    There are a few different ways to do this, I thought this one was
    the most concise. See:
    https://en.wikipedia.org/wiki/Decision_tree_learning#Gini_impurity
    """
    counts = class_counts(rows)
    impurity = 1
    for lbl in counts:
        prob_of_lbl = counts[lbl] / float(len(rows))
        impurity -= prob_of_lbl**2
    return impurity

In [20]:
#######
# Demo:
# Let's look at some example to understand how Gini Impurity works.
#
# First, we'll look at a dataset with no mixing.
no_mixing = [['Apple'],
              ['Apple']]
# this will return 0
gini(no_mixing)

0.0

In [21]:
# Now, we'll look at dataset with a 50:50 apples:oranges ratio
some_mixing = [['Apple'],
               ['Orange']]
# this will return 0.5 - meaning, there's a 50% chance of misclassifying
# a random example we draw from the dataset.
gini(some_mixing)

0.5

In [22]:
# Now, we'll look at a dataset with many different labels
lots_of_mixing = [['Apple'],
                  ['Orange'],
                  ['Grape'],
                  ['Grapefruit'],
                  ['Blueberry']]
# This will return 0.8
gini(lots_of_mixing)
#######

0.7999999999999998

In [23]:
def info_gain(left, right, current_uncertainty):
    """Information Gain.

    The uncertainty of the starting node, minus the weighted impurity of
    two child nodes.
    """
    p = float(len(left)) / (len(left) + len(right))
    return current_uncertainty - p * gini(left) - (1 - p) * gini(right)

In [24]:
#######
# Demo:
# Calculate the uncertainy of our training data.
current_uncertainty = gini(training_data)
current_uncertainty

0.30087170791491236

In [25]:
# How much information do we gain by partioning on 'Green'?
true_rows, false_rows = partition(training_data, Question(1, 19.5, 0, 1))
info_gain(true_rows, false_rows, current_uncertainty)

0.028550543360373576

In [26]:
# What about if we partioned on 'Red' instead?
true_rows, false_rows = partition(training_data, Question(1, 18.1, 1, 0))
info_gain(true_rows, false_rows, current_uncertainty)

0.09618608127219366

In [27]:
# It looks like we learned more using 'Red' (0.37), than 'Green' (0.14).
# Why? Look at the different splits that result, and see which one
# looks more 'unmixed' to you.
true_rows, false_rows = partition(training_data, Question(1, 19.7, 0, 0))

# Here, the true_rows contain only 'Grapes'.
true_rows

[[91.5, 22.3, 'Klassrum'],
 [86.5, 20.7, 'Klassrum'],
 [82.7, 19.8, 'Klassrum'],
 [85.0, 23.6, 'Klassrum'],
 [97.5, 21.4, 'Klassrum'],
 [88.6, 21.5, 'Klassrum'],
 [95.7, 22.8, 'Klassrum'],
 [97.4, 22.6, 'Klassrum'],
 [82.0, 22.7, 'Klassrum'],
 [92.7, 21.8, 'Klassrum'],
 [78.6, 21.9, 'Lärarrum'],
 [92.9, 22.7, 'Klassrum'],
 [90.3, 22.0, 'Klassrum'],
 [78.5, 20.9, 'Lärarrum'],
 [92.4, 23.4, 'Klassrum'],
 [86.7, 22.7, 'Klassrum'],
 [98.6, 20.2, 'Klassrum'],
 [81.7, 23.6, 'Klassrum'],
 [98.1, 22.2, 'Klassrum'],
 [97.2, 21.5, 'Klassrum'],
 [88.0, 20.7, 'Klassrum'],
 [82.8, 22.5, 'Klassrum'],
 [85.5, 22.7, 'Klassrum'],
 [92.2, 19.8, 'Klassrum'],
 [98.5, 21.1, 'Klassrum'],
 [81.1, 21.3, 'Lärarrum'],
 [90.5, 20.6, 'Klassrum'],
 [91.3, 23.9, 'Klassrum'],
 [96.5, 20.7, 'Klassrum'],
 [97.6, 22.7, 'Klassrum'],
 [87.8, 23.6, 'Klassrum'],
 [98.9, 19.9, 'Klassrum'],
 [82.1, 23.8, 'Klassrum'],
 [86.4, 20.3, 'Klassrum'],
 [90.8, 21.8, 'Klassrum'],
 [87.4, 20.2, 'Klassrum'],
 [89.4, 20.1, 'Klassrum'],
 

In [28]:
# And the false rows contain two types of fruit. Not too bad.
false_rows

[[58.0, 15.7, 'Kylrum'],
 [89.8, 18.4, 'Klassrum'],
 [97.4, 18.4, 'Klassrum'],
 [98.3, 19.2, 'Klassrum'],
 [36.0, 10.2, 'Kylrum'],
 [87.2, 19.6, 'Klassrum'],
 [90.0, 18.9, 'Klassrum'],
 [83.5, 18.6, 'Klassrum'],
 [80.7, 19.0, 'Klassrum'],
 [86.5, 19.2, 'Klassrum'],
 [100.0, 19.0, 'Klassrum'],
 [93.1, 18.1, 'Klassrum'],
 [88.8, 18.7, 'Klassrum'],
 [80.1, 19.7, 'Lärarrum'],
 [80.4, 19.2, 'Klassrum'],
 [50.4, 16.2, 'Lärarrum'],
 [70.4, 18.3, 'Lärarrum'],
 [81.0, 18.5, 'Lärarrum'],
 [53.3, 17.2, 'Lärarrum'],
 [87.3, 18.8, 'Klassrum'],
 [98.1, 19.1, 'Klassrum'],
 [96.0, 18.1, 'Klassrum'],
 [97.1, 19.5, 'Klassrum'],
 [90.2, 19.7, 'Klassrum'],
 [70.5, 18.3, 'Lärarrum'],
 [84.7, 19.1, 'Klassrum'],
 [94.9, 18.8, 'Klassrum'],
 [91.3, 18.8, 'Klassrum'],
 [72.4, 17.4, 'Lärarrum'],
 [80.6, 17.6, 'Lärarrum'],
 [69.5, 17.3, 'Lärarrum'],
 [16.1, 12.3, 'Kylrum'],
 [84.7, 19.6, 'Klassrum'],
 [89.2, 19.7, 'Klassrum'],
 [98.8, 19.4, 'Klassrum'],
 [94.4, 18.1, 'Klassrum'],
 [27.7, 11.7, 'Kylrum'],
 [40.8, 

In [29]:
# On the other hand, partitioning by Green doesn't help so much.
true_rows, false_rows = partition(training_data, Question(0, 45.4, 1, 1))

# We've isolated one apple in the true rows.
true_rows

[[36.0, 10.2, 'Kylrum'],
 [16.1, 12.3, 'Kylrum'],
 [27.7, 11.7, 'Kylrum'],
 [40.8, 15.0, 'Kylrum'],
 [21.3, 10.5, 'Kylrum'],
 [25.9, 14.1, 'Kylrum'],
 [41.1, 14.6, 'Kylrum'],
 [35.1, 13.9, 'Kylrum'],
 [21.2, 10.1, 'Kylrum'],
 [31.1, 11.3, 'Kylrum'],
 [27.5, 10.8, 'Kylrum'],
 [35.6, 13.4, 'Kylrum'],
 [18.7, 11.9, 'Kylrum'],
 [31.6, 12.0, 'Kylrum'],
 [30.2, 12.6, 'Kylrum'],
 [21.3, 13.5, 'Kylrum'],
 [42.9, 10.5, 'Kylrum'],
 [31.1, 13.9, 'Kylrum'],
 [42.0, 15.1, 'Kylrum'],
 [28.7, 15.1, 'Kylrum'],
 [39.9, 15.1, 'Kylrum'],
 [30.7, 11.9, 'Kylrum'],
 [45.1, 13.3, 'Kylrum'],
 [40.9, 10.6, 'Kylrum'],
 [27.4, 12.3, 'Kylrum'],
 [34.4, 14.8, 'Kylrum'],
 [19.2, 12.1, 'Kylrum'],
 [20.6, 10.7, 'Kylrum'],
 [18.1, 10.1, 'Kylrum'],
 [42.2, 11.6, 'Kylrum'],
 [34.1, 12.2, 'Kylrum'],
 [27.7, 12.3, 'Kylrum'],
 [35.5, 11.9, 'Kylrum'],
 [39.7, 13.8, 'Kylrum'],
 [26.9, 14.8, 'Kylrum'],
 [16.6, 14.2, 'Kylrum'],
 [31.3, 15.6, 'Kylrum'],
 [18.7, 14.9, 'Kylrum'],
 [38.9, 10.2, 'Kylrum'],
 [41.7, 13.6, 'Kylrum'],


In [30]:
# But, the false-rows are badly mixed up.
false_rows
#######

[[91.5, 22.3, 'Klassrum'],
 [86.5, 20.7, 'Klassrum'],
 [58.0, 15.7, 'Kylrum'],
 [82.7, 19.8, 'Klassrum'],
 [89.8, 18.4, 'Klassrum'],
 [85.0, 23.6, 'Klassrum'],
 [97.5, 21.4, 'Klassrum'],
 [88.6, 21.5, 'Klassrum'],
 [95.7, 22.8, 'Klassrum'],
 [97.4, 18.4, 'Klassrum'],
 [97.4, 22.6, 'Klassrum'],
 [98.3, 19.2, 'Klassrum'],
 [87.2, 19.6, 'Klassrum'],
 [82.0, 22.7, 'Klassrum'],
 [90.0, 18.9, 'Klassrum'],
 [83.5, 18.6, 'Klassrum'],
 [80.7, 19.0, 'Klassrum'],
 [86.5, 19.2, 'Klassrum'],
 [92.7, 21.8, 'Klassrum'],
 [78.6, 21.9, 'Lärarrum'],
 [92.9, 22.7, 'Klassrum'],
 [100.0, 19.0, 'Klassrum'],
 [90.3, 22.0, 'Klassrum'],
 [78.5, 20.9, 'Lärarrum'],
 [92.4, 23.4, 'Klassrum'],
 [93.1, 18.1, 'Klassrum'],
 [86.7, 22.7, 'Klassrum'],
 [98.6, 20.2, 'Klassrum'],
 [88.8, 18.7, 'Klassrum'],
 [81.7, 23.6, 'Klassrum'],
 [80.1, 19.7, 'Lärarrum'],
 [98.1, 22.2, 'Klassrum'],
 [80.4, 19.2, 'Klassrum'],
 [50.4, 16.2, 'Lärarrum'],
 [70.4, 18.3, 'Lärarrum'],
 [97.2, 21.5, 'Klassrum'],
 [88.0, 20.7, 'Klassrum'],
 [

In [31]:
def find_best_split(rows):
    """Find the best question to ask by iterating over every feature / value
    and calculating the information gain."""
    best_gain = 0  # keep track of the best information gain
    best_question = None  # keep train of the feature / value that produced it
    current_uncertainty = gini(rows)
    n_features = len(rows[0]) - 1  # number of columns

    for col in range(n_features):  # for each feature

        values = set([row[col] for row in rows])  # unique values in the column

        for val in values:  # for each value
            for less in range(0,2):
                for eq in range(0,2):
                    question = Question(col, val, less, eq)

                    # try splitting the dataset
                    true_rows, false_rows = partition(rows, question)

                    # Skip this split if it doesn't divide the
                    # dataset.
                    if len(true_rows) == 0 or len(false_rows) == 0:
                        continue

                    # Calculate the information gain from this split
                    gain = info_gain(true_rows, false_rows, current_uncertainty)

                    # You actually can use '>' instead of '>=' here
                    # but I wanted the tree to look a certain way for our
                    # toy dataset.
                    if gain > best_gain:
                        best_gain, best_question = gain, question

    return best_gain, best_question

In [32]:
%%time
#######
# Demo:
# Find the best question to ask first for our toy dataset.
best_gain, best_question = find_best_split(training_data)
best_question
# FYI: is color == Red is just as good. See the note in the code above
# where I used '>='.
#######

Wall time: 4.21 s


Är luftfuktighet <= 79.9?

In [34]:
class Leaf:
    """A Leaf node classifies data.

    This holds a dictionary of class (e.g., "Apple") -> number of times
    it appears in the rows from the training data that reach this leaf.
    """

    def __init__(self, rows):
        self.predictions = class_counts(rows)

In [35]:
class Decision_Node:
    """A Decision Node asks a question.

    This holds a reference to the question, and to the two child nodes.
    """

    def __init__(self,
                 question,
                 true_branch,
                 false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

In [36]:
def build_tree(rows):
    """Builds the tree.

    Rules of recursion: 1) Believe that it works. 2) Start by checking
    for the base case (no further information gain). 3) Prepare for
    giant stack traces.
    """

    # Try partitioing the dataset on each of the unique attribute,
    # calculate the information gain,
    # and return the question that produces the highest gain.
    gain, question = find_best_split(rows)

    # Base case: no further info gain
    # Since we can ask no further questions,
    # we'll return a leaf.
    if gain == 0:
        return Leaf(rows)

    # If we reach here, we have found a useful feature / value
    # to partition on.
    true_rows, false_rows = partition(rows, question)

    # Recursively build the true branch.
    true_branch = build_tree(true_rows)

    # Recursively build the false branch.
    false_branch = build_tree(false_rows)

    # Return a Question node.
    # This records the best feature / value to ask at this point,
    # as well as the branches to follow
    # dependingo on the answer.
    return Decision_Node(question, true_branch, false_branch)

In [37]:
def print_tree(node, spacing=""):
    """World's most elegant tree printing function."""

    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        print (spacing + "Predict", node.predictions)
        return

    # Print the question at this node
    print (spacing + str(node.question))

    # Call this function recursively on the true branch
    print (spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")

    # Call this function recursively on the false branch
    print (spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")

In [38]:
## Här nedan är decisionForest funktionen
## Även en loop som bygger 20 träd från 500 slumpmässigt valda datapunkter


In [42]:
%%time
trees = []
for n in range (1,21):
    tree = build_tree(random.sample(training_data, 500))
    trees.append(tree)
def decisionForest(row):
    votes = {'Kylrum': 0, 'Lärarrum': 0, 'Klassrum': 0}
    for tree in trees:
        predict = classify(row, tree)
        maxValue = max(predict.items(), key = operator.itemgetter(1))[0]
        votes[maxValue] += 1/len(trees)
    for key in votes:
        votes[key] = str(round(votes[key]*100)) + '%'
    return votes
decisionForest([30, 15.6, 'Kylrum'])

Wall time: 46.5 s


{'Kylrum': '100%', 'Lärarrum': '0%', 'Klassrum': '0%'}

In [46]:
my_tree = build_tree(training_data)

In [49]:
print_tree(my_tree)

Är temperatur >= 16.0?
--> True:
  Är luftfuktighet > 90.0?
  --> True:
    Predict {'Klassrum': 143}
  --> False:
    Är temperatur > 22.0?
    --> True:
      Predict {'Klassrum': 50}
    --> False:
      Är luftfuktighet > 80.1?
      --> True:
        Är temperatur >= 18.1?
        --> True:
          Är luftfuktighet >= 88.0?
          --> True:
            Är luftfuktighet < 88.7?
            --> True:
              Är temperatur >= 21.5?
              --> True:
                Är temperatur >= 21.9?
                --> True:
                  Predict {'Lärarrum': 2}
                --> False:
                  Predict {'Klassrum': 1}
              --> False:
                Är luftfuktighet >= 88.3?
                --> True:
                  Predict {'Lärarrum': 9}
                --> False:
                  Är luftfuktighet >= 88.2?
                  --> True:
                    Är temperatur >= 20.5?
                    --> True:
                      Predict {'Lärarrum': 1

                              Predict {'Lärarrum': 2}
                  --> False:
                    Är luftfuktighet > 84.4?
                    --> True:
                      Är luftfuktighet > 84.7?
                      --> True:
                        Är temperatur > 21.1?
                        --> True:
                          Predict {'Lärarrum': 2}
                        --> False:
                          Predict {'Lärarrum': 1, 'Klassrum': 1}
                      --> False:
                        Är luftfuktighet > 84.5?
                        --> True:
                          Predict {'Klassrum': 2}
                        --> False:
                          Är temperatur > 21.1?
                          --> True:
                            Predict {'Klassrum': 1}
                          --> False:
                            Predict {'Lärarrum': 1}
                    --> False:
                      Är luftfuktighet >= 83.2?
                      --> Tr

In [43]:
def classify(row, node):
    """See the 'rules of recursion' above."""

    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        return node.predictions

    # Decide whether to follow the true-branch or the false-branch.
    # Compare the feature / value stored in the node,
    # to the example we're considering.
    if node.question.moreless(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)

In [44]:
def print_leaf(counts):
    """A nicer way to print the predictions at a leaf."""
    total = sum(counts.values())
    probs = {}
    for lbl in counts.keys():
        probs[lbl] = str(int(counts[lbl] / total * 100)) + "%"
    return probs

In [45]:
#######
# Demo:
# Printing that a bit nicer
print_leaf(classify(training_data[0], my_tree))
#######

NameError: name 'my_tree' is not defined

In [54]:
#######
# Demo:
# On the second example, the confidence is lower
print_leaf(classify(training_data[1], my_tree))
#######

{'Klassrum': '100%'}

In [47]:
# Evaluate
testing_data = dataGenerator(5)

In [56]:
## Här nedan körs decisionForest funktionen på lite test-data

In [51]:
for row in testing_data:
    print ("Actual: %s. Predicted: %s" %
           (row[-1], decisionForest(row)))

Actual: Kylrum. Predicted: {'Kylrum': '100%', 'Lärarrum': '0%', 'Klassrum': '0%'}
Actual: Kylrum. Predicted: {'Kylrum': '100%', 'Lärarrum': '0%', 'Klassrum': '0%'}
Actual: Lärarrum. Predicted: {'Kylrum': '0%', 'Lärarrum': '5%', 'Klassrum': '95%'}
Actual: Lärarrum. Predicted: {'Kylrum': '0%', 'Lärarrum': '15%', 'Klassrum': '85%'}
Actual: Lärarrum. Predicted: {'Kylrum': '0%', 'Lärarrum': '100%', 'Klassrum': '0%'}
Actual: Klassrum. Predicted: {'Kylrum': '0%', 'Lärarrum': '0%', 'Klassrum': '100%'}
Actual: Klassrum. Predicted: {'Kylrum': '0%', 'Lärarrum': '0%', 'Klassrum': '100%'}
Actual: Klassrum. Predicted: {'Kylrum': '0%', 'Lärarrum': '0%', 'Klassrum': '100%'}
